# python libraries


In [1]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")
# coding=utf-8
import re
import string
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import xml.etree.cElementTree as ET
import io
import pandas as pd
pd.set_option('display.max_colwidth', -1)

import demoji
import xml.etree.cElementTree as ET
import urllib.request
import emoji
from emoji import UNICODE_EMOJI
from emoji import emojize
from googletrans import Translator
from google_trans_new import google_translator
from collections import Counter
import json
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import RegexpTokenizer
import requests
from lxml import html
import matplotlib.pyplot as plt
import pandas
import seaborn as sns 
from sklearn import preprocessing
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from collections import Counter 
import re
import string
import matplotlib.cm as cm
from matplotlib import rcParams
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

open dataset

In [2]:
#dataframe 
file_name_output = "listestatut.csv"
with  io.open(file_name_output, encoding="utf-8") as f:
      lines = f.read().splitlines()

df =pd.DataFrame([mot for mot in lines] , columns=['arab'])

Data Pre-processing


In [3]:
for letter in '@#.][!XR':
    df['arab'] = df['arab'].astype(str).str.replace(letter,'')

In [4]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

Normalization

In [5]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

Remove Repetited characters



In [6]:
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

apply all functions



In [7]:
import re
#convert to lowercase, strip and remove punctuations

def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    #Replace @username with empty string
    text = re.sub('@[^\s]+', ' ', text)
    #Convert www.* or https?://* to " "
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)
    #Replace #word with word
    text = re.sub(r'#([^\s]+)', r'\1', text)
    # remove punctuations
    text= remove_punctuations(text)
    # normalize the comment
    text= normalize_arabic(text)
    # remove repeated letters
    text=remove_repeating_char(text)
    
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text=re.compile( r"(.)\1+" ).sub( r"\1", text )
    # remove hyperlinks
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    text = re.sub(r'http?:\/\/.*[\r\n]*', '', text)
    #remove hashtag sign
    #text=re.sub(r"#","",text)   
    #remove mentions
    text = re.sub(r"(?:\@)\w+", '', text)
    text=re.sub(r'[:"#$%&\*+,-/:;<=>@\\^_`{|}~]+','',text)
    text=re.sub(r'[!]+','!',text)
    text=re.sub(r'[?]+','?',text)
    text=re.sub(r'[.]+','.',text)
    text=re.sub(r"'","",text)
    text=re.sub(r"\(","",text)
    text=re.sub(r"\)","",text)
    
    text=" ".join(text.split())
    return text

In [8]:
df["arab"] = df['arab'].apply(lambda x: preprocess(x)) # apply used to call the method processpost
df


,arab
0,kamel fourati msg prv mr
1,نحب وحده كيفاش نعمل
2,samir ben moalem envoyé nous mp
3,نحب نشري
4,ahlem ben abdalah msg prv madame
...,...
30888,hayet ghnimib9adech palete ili kont twari fiha
30889,abia bnhsanبقداش مسك الطهاره و فخر الطافه
30890,mimou mariemvictoria 9adeh
30891,abeb lhamemiخليهالي si 2678282


Remove duplications



In [9]:
#removing
df.drop_duplicates(inplace=True)

#checking if still contain duplications
df.duplicated().sum()
df
df.to_csv("new.csv", encoding='utf-8')

-----------------------------------------------------------------------

Tokenize the comments text



In [10]:
tokenizer = RegexpTokenizer(r'\w+')
df["arab"] = df["arab"].apply(tokenizer.tokenize)

In [11]:
df["arab"].head(10)


0    [kamel, fourati, msg, prv, mr]         
1    [نحب, وحده, كيفاش, نعمل]               
2    [samir, ben, moalem, envoyé, nous, mp] 
3    [نحب, نشري]                            
4    [ahlem, ben, abdalah, msg, prv, madame]
5    [نحب, نشري, كعبه]                      
6    [jemjem, jemjem, msg, prv, mr]         
7    [نحب, نشري, وحده]                      
8    [samira, fouchali, msg, prv, madame]   
9    [نحب, نشري, وحدا]                      
Name: arab, dtype: object

Remove Stop words (ARABIC/ENGLISH/FRENCH)



In [12]:
import nltk
nltk.download('stopwords')
stopwords_listArabic = stopwords.words('arabic') 
stopwords_listArabic

[nltk_data] Downloading package stopwords to C:\Users\YOUSOFT
[nltk_data]     PC1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['إذ',
 'إذا',
 'إذما',
 'إذن',
 'أف',
 'أقل',
 'أكثر',
 'ألا',
 'إلا',
 'التي',
 'الذي',
 'الذين',
 'اللاتي',
 'اللائي',
 'اللتان',
 'اللتيا',
 'اللتين',
 'اللذان',
 'اللذين',
 'اللواتي',
 'إلى',
 'إليك',
 'إليكم',
 'إليكما',
 'إليكن',
 'أم',
 'أما',
 'أما',
 'إما',
 'أن',
 'إن',
 'إنا',
 'أنا',
 'أنت',
 'أنتم',
 'أنتما',
 'أنتن',
 'إنما',
 'إنه',
 'أنى',
 'أنى',
 'آه',
 'آها',
 'أو',
 'أولاء',
 'أولئك',
 'أوه',
 'آي',
 'أي',
 'أيها',
 'إي',
 'أين',
 'أين',
 'أينما',
 'إيه',
 'بخ',
 'بس',
 'بعد',
 'بعض',
 'بك',
 'بكم',
 'بكم',
 'بكما',
 'بكن',
 'بل',
 'بلى',
 'بما',
 'بماذا',
 'بمن',
 'بنا',
 'به',
 'بها',
 'بهم',
 'بهما',
 'بهن',
 'بي',
 'بين',
 'بيد',
 'تلك',
 'تلكم',
 'تلكما',
 'ته',
 'تي',
 'تين',
 'تينك',
 'ثم',
 'ثمة',
 'حاشا',
 'حبذا',
 'حتى',
 'حيث',
 'حيثما',
 'حين',
 'خلا',
 'دون',
 'ذا',
 'ذات',
 'ذاك',
 'ذان',
 'ذانك',
 'ذلك',
 'ذلكم',
 'ذلكما',
 'ذلكن',
 'ذه',
 'ذو',
 'ذوا',
 'ذواتا',
 'ذواتي',
 'ذي',
 'ذين',
 'ذينك',
 'ريث',
 'سوف',
 'سوى',
 'شتان',
 'عدا',
 'عسى',
 'عل'

In [13]:
stopwords_listEnglish = stopwords.words('english')
stopwords_listEnglish


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [14]:
stopwords_listFrench = stopwords.words('french')
stopwords_listFrench

['au',
 'aux',
 'avec',
 'ce',
 'ces',
 'dans',
 'de',
 'des',
 'du',
 'elle',
 'en',
 'et',
 'eux',
 'il',
 'ils',
 'je',
 'la',
 'le',
 'les',
 'leur',
 'lui',
 'ma',
 'mais',
 'me',
 'même',
 'mes',
 'moi',
 'mon',
 'ne',
 'nos',
 'notre',
 'nous',
 'on',
 'ou',
 'par',
 'pas',
 'pour',
 'qu',
 'que',
 'qui',
 'sa',
 'se',
 'ses',
 'son',
 'sur',
 'ta',
 'te',
 'tes',
 'toi',
 'ton',
 'tu',
 'un',
 'une',
 'vos',
 'votre',
 'vous',
 'c',
 'd',
 'j',
 'l',
 'à',
 'm',
 'n',
 's',
 't',
 'y',
 'été',
 'étée',
 'étées',
 'étés',
 'étant',
 'étante',
 'étants',
 'étantes',
 'suis',
 'es',
 'est',
 'sommes',
 'êtes',
 'sont',
 'serai',
 'seras',
 'sera',
 'serons',
 'serez',
 'seront',
 'serais',
 'serait',
 'serions',
 'seriez',
 'seraient',
 'étais',
 'était',
 'étions',
 'étiez',
 'étaient',
 'fus',
 'fut',
 'fûmes',
 'fûtes',
 'furent',
 'sois',
 'soit',
 'soyons',
 'soyez',
 'soient',
 'fusse',
 'fusses',
 'fût',
 'fussions',
 'fussiez',
 'fussent',
 'ayant',
 'ayante',
 'ayantes',


In [15]:
df["arab"]=df["arab"].apply(lambda x: [item for item in x if item not in stopwords_listArabic])


In [16]:
df["arab"]=df["arab"].apply(lambda x: [item for item in x if item not in stopwords_listFrench])


In [17]:
df["arab"]=df["arab"].apply(lambda x: [item for item in x if item not in stopwords_listEnglish])


In [18]:
all_words = [word for tokens in df["arab"] for word in tokens]
sentence_lengths = [len(tokens) for tokens in df["arab"]]

VOCAB = sorted(list(set(all_words)))

print("%s words total, with a vocabulary size of %s" % (len(all_words), len(VOCAB)))
print("Max sentence length is %s" % max(sentence_lengths))

108611 words total, with a vocabulary size of 27446
Max sentence length is 212


In [19]:
counter = Counter(all_words)
counter.most_common(10)

[('w', 1223),
 ('نحب', 964),
 ('taile', 910),
 ('بيك', 796),
 ('الي', 764),
 ('لوكسولور', 757),
 ('الخدمه', 734),
 ('عالمسنجر', 729),
 ('التواصل', 728),
 ('لتسهيل', 727)]